# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geo_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto lempira,15.2667,-83.7667,30.08,66,75,0.00,HN,1717950955
1,1,chibuto,-24.6867,33.5306,17.16,67,94,2.17,MZ,1717950955
2,2,imbituba,-28.2400,-48.6703,24.40,57,24,5.60,BR,1717950956
3,3,port-aux-francais,-49.3500,70.2167,6.72,98,100,17.72,TF,1717950872
4,4,tiksi,71.6872,128.8694,0.12,96,96,2.46,RU,1717950875


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
#%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    frame_width = 700,
    frame_height = 500,
    scale = 0.8,
    size = "Humidity",
    color = "City",
    tiles = "OSM",
    hover_cols = ["Lat", "Lng","Humidity", "City"]
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
weather_condition = city_data_df.loc[(city_data_df["Max Temp"]>18)&(city_data_df["Max Temp"]<30)]
weather_condition = city_data_df.loc[city_data_df["Cloudiness"] == 0]

# Drop any rows with null values
weather_condition = weather_condition.dropna()

# Display sample data
weather_condition

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
9,9,al jawf,29.5000,38.7500,34.69,10,0,4.84,SA,1717950956
16,16,ceres,37.5949,-120.9577,23.45,49,0,0.00,US,1717950957
21,21,dawesville,30.9224,-84.0091,33.53,39,0,3.09,US,1717950958
30,30,moranbah,-22.0016,148.0466,11.32,56,0,2.47,AU,1717950958
35,35,laguna,38.4210,-121.4238,23.51,60,0,4.12,US,1717950882
...,...,...,...,...,...,...,...,...,...,...
546,546,kupang,-10.1667,123.5833,25.32,94,0,2.57,ID,1717951014
558,558,edna,28.9786,-96.6461,33.95,38,0,5.66,US,1717951015
559,559,gloucester point,37.2540,-76.4969,32.21,48,0,6.69,US,1717951015
574,574,as saffaniyah,27.9708,48.7300,31.49,62,0,4.36,SA,1717951017


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = weather_condition
hotel_df["Hotel Name"] = ""

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE

# Display sample data
hotel_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
9,9,al jawf,29.5000,38.7500,34.69,10,0,4.84,SA,1717950956,
16,16,ceres,37.5949,-120.9577,23.45,49,0,0.00,US,1717950957,
21,21,dawesville,30.9224,-84.0091,33.53,39,0,3.09,US,1717950958,
30,30,moranbah,-22.0016,148.0466,11.32,56,0,2.47,AU,1717950958,
35,35,laguna,38.4210,-121.4238,23.51,60,0,4.12,US,1717950882,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
#"https://api.geoapify.com/v2/places?categories=accommodation.hotel&filter=circle:-79.516221,43.706359,1000&bias=proximity:-79.516221,43.706359&limit=20&apiKey=YOUR_API_KEY"
radius = 10000
categories = "accommodation.hotel"

params = {
    "categories":categories,
    "apiKey":geo_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, col in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = col["Lat"]
    longitude = col["Lng"]
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
al jawf - nearest hotel: No hotel found
ceres - nearest hotel: Howard Johnson by Wyndham Modesto Ceres
dawesville - nearest hotel: Baymont by Wyndham Thomasville
moranbah - nearest hotel: No hotel found
laguna - nearest hotel: Holiday Inn Express & Suites
port alfred - nearest hotel: No hotel found
hamilton - nearest hotel: North Vista Manor
karratha - nearest hotel: Karratha International Hotel
namtsy - nearest hotel: No hotel found
cabo san lucas - nearest hotel: Comfort Rooms
plettenberg bay - nearest hotel: Milkwood Manor
port elizabeth - nearest hotel: Waterford Hotel
arar - nearest hotel: دار أوتيل Dar Otel
east london - nearest hotel: No hotel found
argostolion - nearest hotel: Ainos Hotel
mount pleasant - nearest hotel: Old Village Post House
plattsmouth - nearest hotel: No hotel found
keflavik - nearest hotel: Núpan Deluxe
waddan - nearest hotel: No hotel found
selma - nearest hotel: Best Western
kenai - nearest hotel: Beluga Lookout Hotel
brookings - nea

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
9,9,al jawf,29.5000,38.7500,34.69,10,0,4.84,SA,1717950956,No hotel found
16,16,ceres,37.5949,-120.9577,23.45,49,0,0.00,US,1717950957,Howard Johnson by Wyndham Modesto Ceres
21,21,dawesville,30.9224,-84.0091,33.53,39,0,3.09,US,1717950958,Baymont by Wyndham Thomasville
30,30,moranbah,-22.0016,148.0466,11.32,56,0,2.47,AU,1717950958,No hotel found
35,35,laguna,38.4210,-121.4238,23.51,60,0,4.12,US,1717950882,Holiday Inn Express & Suites
...,...,...,...,...,...,...,...,...,...,...,...
546,546,kupang,-10.1667,123.5833,25.32,94,0,2.57,ID,1717951014,HOTEL SALUNGA
558,558,edna,28.9786,-96.6461,33.95,38,0,5.66,US,1717951015,No hotel found
559,559,gloucester point,37.2540,-76.4969,32.21,48,0,6.69,US,1717951015,Yorktown Beach Hotel
574,574,as saffaniyah,27.9708,48.7300,31.49,62,0,4.36,SA,1717951017,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:

# Configure the map plot
map_plot_hotel = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    frame_width = 700,
    frame_height = 500,
    scale = 0.8,
    color = "City",
    tiles = "OSM",
    hover_cols = ["Lat", "Lng","Humidity", "City", "Hotel Name", "Country"]
)

# Display the map
map_plot_hotel

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)